# Agent Onboarding buddy

This notebook uses the following agents:

1. __User proxy__: Represents the human user who provides an initial prompt to the agent - in our case, a new hire 
2. __Onboarding buddy__: 
3. __Supervisor__: 

In [13]:
from dotenv import load_dotenv, find_dotenv

import autogen
from autogen.cache import Cache
import os

load_dotenv(find_dotenv())

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST.json",
    filter_dict={
        "model": ["gpt-4o", "gpt4-turbo"],
    },
)

settings = {
    'host': os.getenv('ACCOUNT_HOST'),
    'master_key': os.getenv('ACCOUNT_KEY'),
    'database_id': os.getenv('COSMOS_DATABASE'),
    'container_id': os.getenv('COSMOS_CONTAINER_EMPLOYEE'),
    'search_endpoint': os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT'),
    'serch_key': os.getenv('AZURE_SEARCH_KEY'),
    'search_api_version': os.getenv('AZURE_SEARCH_API_VERSION')
}


for key, value in settings.items():
    if value is None:
        raise ValueError(f"Missing environment variable for {key}")

In [14]:
onboarding_buddy_prompt = """
You are an experienced onboarding buddy, working with a new hire. You need to guide the user through the onboarding process. You have all tthe information you need to help the user onboard. You are responsible for ensuring the user is onboarded successfully.
If you do not know the answer to a question, you can ask the supervisor for help. You can retrieve the employee information from the database based on the Employee ID, and retrieve information from the knowledge base by providing the search terms to the function "search".  
"""

supervisor_prompt = """
You are a supervisor.You are responsible for the onboarding process and ensuring that the new hire is onboarded successfully. 
After the onboarding buddy has guided the new hire through the onboarding process, you need to verify that the new hire has been onboarded properly by asking him a few test questions.
"""

In [15]:
import tempfile
from autogen import ConversableAgent
from autogen.coding import DockerCommandLineCodeExecutor

llm_config = {"config_list": config_list, "cache_seed": 42}
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="The new hire.",
    human_input_mode="ALWAYS",
    code_execution_config=False
)

onboarding_buddy = autogen.ConversableAgent(
    name="onboarding_buddy",
    system_message= onboarding_buddy_prompt,
    llm_config=llm_config,
)
supervisor = autogen.ConversableAgent(
    name="supervisor",
    system_message=supervisor_prompt,
    llm_config=llm_config,
)

temp_dir = tempfile.TemporaryDirectory()

executor = DockerCommandLineCodeExecutor(
    image="python:3.12-slim",  
    timeout=10,  
    work_dir=temp_dir.name
)


code_executor_agent_using_docker = ConversableAgent(
    "code_executor_agent_docker",
    llm_config=False, 
    code_execution_config={"executor": executor},  
    human_input_mode="NEVER", 
)

groupchat = autogen.GroupChat(agents=[user_proxy, onboarding_buddy, supervisor, code_executor_agent_using_docker], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)


In [16]:
import json
import requests
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
import json

HOST = settings['host']
MASTER_KEY = settings['master_key']
DATABASE_ID = settings['database_id']
CONTAINER_ID = settings['container_id']

client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY})
db = client.get_database_client(DATABASE_ID)
container = db.get_container_client(CONTAINER_ID)

@code_executor_agent_using_docker.register_for_execution()
@onboarding_buddy.register_for_llm(name="get_employee_info", description="Retrieve employee data from the database, including the start date and mandatory exam asignments.")
def get_employee_info(EmployeeId: str) -> str:
    try:
        query = f"SELECT * FROM c WHERE c.EmployeeId = '{EmployeeId}'"
        items = list(container.query_items(query=query, enable_cross_partition_query=True))
        
        if items:
            employee_info = items[0]
            return json.dumps(employee_info, default=str)
        else:
            error_message = {'error': f"Employee with ID '{EmployeeId}' not found."}
            return json.dumps(error_message)
    
    except exceptions.CosmosHttpResponseError as e:
        error_message = {'error': str(e)}
        return json.dumps(error_message)

In [17]:
from openai import AzureOpenAI
from typing import List, Dict
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import (
    VectorizedQuery,
    QueryType,
    QueryCaptionType,
    QueryAnswerType,
)
import os

def get_embedding(query: str) -> List[float]:
    client = AzureOpenAI(
        api_key=os.getenv("AZURE_OAI_KEY"),
        azure_endpoint=os.getenv("AZURE_OAI_ENDPOINT"),
        api_version="2024-02-01"
    )
    try:
        response = client.embeddings.create(
            input=query,
            model="embedding",
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Error getting embedding: {e}")
        return []

def retrieve_documentation(query: str) -> List[Dict]:
    try:
        embedding = get_embedding(query)
        if not embedding:
            return []

        search_client = SearchClient(
            endpoint=os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"],
            index_name="onboarding",
            credential=AzureKeyCredential(os.getenv("AZURE_SEARCH_KEY"))
        )

        vector_query = VectorizedQuery(
            vector=embedding, k_nearest_neighbors=3, fields="contentVector"
        )

        results = search_client.search(
            search_text=query,
            vector_queries=[vector_query],
            query_type=QueryType.SEMANTIC,
            semantic_configuration_name="azureml-default",
            query_caption=QueryCaptionType.EXTRACTIVE,
            query_answer=QueryAnswerType.EXTRACTIVE,
            top=3,
        )

        docs = [
            {
                "id": doc["id"],
                "title": doc["title"],
                "content": doc["content"],
                "url": doc["url"],
            }
            for doc in results
        ]

        return docs
    except Exception as e:
        print(f"Error retrieving documentation: {e}")
        return []

@code_executor_agent_using_docker.register_for_execution()
@onboarding_buddy.register_for_llm(name="search", description="Search the knowledge base for documentation to answer the question of the user.")
def search(query: str) -> List[Dict]:
    return retrieve_documentation(query)

In [18]:
print(search("Code of Conduct Contoso"))


[{'id': 'b25ib2FyZDMuZG9jeDA=', 'title': '', 'content': "Title: onboard3.docx\n\nContoso Code of Conduct and Key Policies\nWelcome to Contoso! As a new member of our team, it's crucial that you familiarize yourself with our Code of Conduct and key policies. These guidelines form the foundation of our corporate culture and help ensure a safe, respectful, and productive work environment for all.\n1. Code of Conduct\nOur Code of Conduct outlines the behaviors and practices we expect from all Contoso employees:\na) Respect and Inclusion:\n· Treat all colleagues, clients, and partners with respect and dignity.\n· Embrace diversity and foster an inclusive environment.\n· Zero tolerance for discrimination, harassment, or bullying of any kind.\nb) Integrity and Honesty:\n· Conduct business with the highest level of integrity.\n· Be truthful in all communications, both internal and external.\n· Avoid any actions that could be perceived as deceptive or fraudulent.\nc) Compliance with Laws and Re

In [19]:
user_proxy.initiate_chat(
    manager, message="Hi! I'm a new employee."
)

User_proxy (to chat_manager):

Hi! I'm a new employee.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
onboarding_buddy (to chat_manager):

Welcome to the team! I'm here to help you get started with your onboarding process. Could you please provide me with your Employee ID so I can retrieve your information from our database?

--------------------------------------------------------------------------------
User_proxy (to chat_manager):

123

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
onboarding_buddy (to chat_manager):

***** Suggested tool call (call_utNb1HqsJy4qB4NulkhuXZ0L): get_employee_info *****
Arguments: 
{"EmployeeId":"123"}
**********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_employee_info...
code_execut

ChatResult(chat_id=None, chat_history=[{'content': "Hi! I'm a new employee.", 'role': 'assistant'}, {'content': "Welcome to the team! I'm here to help you get started with your onboarding process. Could you please provide me with your Employee ID so I can retrieve your information from our database?", 'name': 'onboarding_buddy', 'role': 'user'}, {'content': '123', 'role': 'assistant'}, {'content': '', 'tool_calls': [{'id': 'call_utNb1HqsJy4qB4NulkhuXZ0L', 'function': {'arguments': '{"EmployeeId":"123"}', 'name': 'get_employee_info'}, 'type': 'function'}], 'name': 'onboarding_buddy', 'role': 'assistant'}, {'content': '{"id": "1", "EmployeeId": "123", "Role": "Procurement specialist", "DaysInRole": "2", "EquipmentReceived": "yes", "ExternalHire": "yes", "OnboardedToITsystems": "yes", "MandatoryExam": "ETHICAL_PROCUREMENT at Contoso, Code of Conduct", "DeadlineForExam": "10 days in the role", "_rid": "ouUXANdCcWABAAAAAAAAAA==", "_self": "dbs/ouUXAA==/colls/ouUXANdCcWA=/docs/ouUXANdCcWABAA